In [4]:
import numpy as np
import pandas as pd
from datetime import datetime


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
import lightgbm as lgb

from sklearn.metrics import mean_squared_error

In [18]:
data_automobile = pd.read_csv('../Data collection/initial_datasets/automobile_data.csv')
backup_data = pd.read_csv("../Data collection/initial_datasets/automobile_data.csv")

In [19]:
data_automobile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1656 entries, 0 to 1655
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Titre               1656 non-null   object
 1   Prix                1656 non-null   object
 2   Fabricant           1656 non-null   object
 3   Carrosserie         1656 non-null   object
 4   Energie             1656 non-null   object
 5   Puissance fiscale   1656 non-null   object
 6   Boite               1656 non-null   object
 7   Transmission        1656 non-null   object
 8   Couleur             1656 non-null   object
 9   Kilometrage         1656 non-null   object
 10  Annee               1656 non-null   object
 11  Gouvernorat         1656 non-null   object
 12  Numero              1656 non-null   object
 13  Inserée le          1656 non-null   object
 14  Number of Pictures  1656 non-null   int64 
 15  Description         1215 non-null   object
dtypes: int64(1), object(15)


In [20]:
data_automobile.isna( ).sum()

Titre                   0
Prix                    0
Fabricant               0
Carrosserie             0
Energie                 0
Puissance fiscale       0
Boite                   0
Transmission            0
Couleur                 0
Kilometrage             0
Annee                   0
Gouvernorat             0
Numero                  0
Inserée le              0
Number of Pictures      0
Description           441
dtype: int64

In [21]:
null_columns = data_automobile.columns[data_automobile.isna().mean() > 0.25]
data_automobile = data_automobile.drop(null_columns, axis=1)

In [22]:
data_automobile.isna().sum()

Titre                 0
Prix                  0
Fabricant             0
Carrosserie           0
Energie               0
Puissance fiscale     0
Boite                 0
Transmission          0
Couleur               0
Kilometrage           0
Annee                 0
Gouvernorat           0
Numero                0
Inserée le            0
Number of Pictures    0
dtype: int64

In [43]:
data_automobile.drop('Number of Pictures'	, axis=1)

,Prix,Fabricant,Carrosserie,Energie,Puissance fiscale,Boite,Transmission,Kilometrage,Age,Posting Month,Posting Year
0,23900.0,CHEVROLET,Berline,Essence,6.0,Manuelle,Traction,216000.0,17,December,2023
1,58000.0,MERCEDES,SUV/4x4,Diesel,15.0,Automatique,Traction,266000.0,18,December,2023
2,85000.0,RENAULT,SUV/4x4,Essence,7.0,Automatique,Traction,85000.0,4,December,2023
3,55000.0,HYUNDAI,SUV/4x4,Diesel,11.0,Automatique,Traction,309000.0,11,December,2023
4,30500.0,PEUGEOT,Citadine,Essence,5.0,Manuelle,Traction,97000.0,9,December,2023
...,...,...,...,...,...,...,...,...,...,...,...
1651,48500.0,VOLKSWAGEN,Berline,Essence,5.0,Manuelle,Traction,158000.0,12,November,2023
1652,110000.0,MERCEDES,Berline,Essence,10.0,Automatique,Propulsion,130000.0,10,November,2023
1653,115000.0,KIA,SUV/4x4,Hybride (diesel),8.0,Automatique,Traction,75000.0,4,November,2023
1654,58000.0,MAZDA,Berline,Essence,6.0,Manuelle,Traction,113000.0,7,December,2023


In [24]:
data_automobile = data_automobile.drop_duplicates()

drop the colums with many unique values

In [25]:
{column: len(data_automobile[column].unique()) for column in data_automobile.columns if data_automobile.dtypes[column] == 'object'}

{'Titre': 978,
 'Prix': 363,
 'Fabricant': 50,
 'Carrosserie': 10,
 'Energie': 5,
 'Puissance fiscale': 34,
 'Boite': 2,
 'Transmission': 3,
 'Couleur': 14,
 'Kilometrage': 401,
 'Annee': 253,
 'Gouvernorat': 23,
 'Numero': 774,
 'Inserée le': 38}

In [26]:
data_automobile = data_automobile.drop(['Titre','Numero','Couleur'], axis=1)

In [27]:
shape = data_automobile.shape
print(f"The shape of the dataframe is {shape[0]} rows and {shape[1]} columns")

The shape of the dataframe is 1656 rows and 12 columns


In [ ]:
# Extract the numeric part
data_automobile['Puissance fiscale'] = data_automobile['Puissance fiscale'].str.extract('(\d+)')

# Convert to integer
data_automobile['Puissance fiscale'] = data_automobile['Puissance fiscale'].astype(int)

In [29]:
data_automobile['Puissance fiscale'] = data_automobile['Puissance fiscale'].astype(float)

In [30]:
# Remove spaces and convert to float
data_automobile['Prix'] = data_automobile['Prix'].str.replace(' ', '').astype(float)

In [31]:
data_automobile['Kilometrage'] = (
    data_automobile['Kilometrage']
    .str.replace(' km', '', regex=False)  # Remove ' km'
    .str.replace(' ', '')                # Remove spaces
    .astype(float)                       # Convert to float
)

In [32]:
current_date = datetime.now()

data_automobile['Age'] = data_automobile['Annee'].apply(lambda x: (current_date.year - int(x.split('-')[1])) +
                                         (current_date.month - int(x.split('-')[0])) / 12)

In [33]:
data_automobile['Age'] = data_automobile['Age'].round().astype(int)

In [34]:
data_automobile = data_automobile.drop('Annee', axis=1)

In [35]:
data_automobile[['Day', 'Posting Month', 'Posting Year']] = data_automobile['Inserée le'].str.split('.', expand=True)

In [36]:
data_automobile = data_automobile.drop('Day', axis=1)
data_automobile['Posting Month'] = data_automobile['Posting Month'].astype(int)
data_automobile['Posting Year'] = data_automobile['Posting Year'].astype(int)
data_automobile = data_automobile.drop('Inserée le', axis=1)

In [37]:
month_names = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

data_automobile['Posting Month'] = data_automobile['Posting Month'].map(month_names)

In [38]:
incorrect_prices = (data_automobile['Prix'] < 100) & (data_automobile['Prix'] > 10)

data_automobile.loc[incorrect_prices, 'Prix'] *= 1000
data_automobile = data_automobile[data_automobile['Prix'].apply(lambda x: len(set(str(x))) != 1) & data_automobile['Prix'].between(10000, 800000)]

In [39]:
# Remove rows with negative mileage
data_automobile = data_automobile[data_automobile['Kilometrage'] >= 0]

# Remove rows where mileage <= 50,000 and the posting year is earlier than 2018
data_automobile = data_automobile[~((data_automobile['Kilometrage'] <= 50000) & (data_automobile['Posting Year'] < 2018))]

# Keep rows with mileage < 1 (e.g., new cars) or realistic mileage between 1,001 and 650,000
data_automobile = data_automobile[(data_automobile['Kilometrage'] < 1) | data_automobile['Kilometrage'].between(1001, 650000)]


In [40]:
data_automobile['Fabricant'] = data_automobile['Fabricant'].str.upper()


In [42]:
data_automobile = data_automobile.drop('Gouvernorat', axis=1)

In [44]:
data_automobile

,Prix,Fabricant,Carrosserie,Energie,Puissance fiscale,Boite,Transmission,Kilometrage,Number of Pictures,Age,Posting Month,Posting Year
0,23900.0,CHEVROLET,Berline,Essence,6.0,Manuelle,Traction,216000.0,12,17,December,2023
1,58000.0,MERCEDES,SUV/4x4,Diesel,15.0,Automatique,Traction,266000.0,12,18,December,2023
2,85000.0,RENAULT,SUV/4x4,Essence,7.0,Automatique,Traction,85000.0,12,4,December,2023
3,55000.0,HYUNDAI,SUV/4x4,Diesel,11.0,Automatique,Traction,309000.0,12,11,December,2023
4,30500.0,PEUGEOT,Citadine,Essence,5.0,Manuelle,Traction,97000.0,12,9,December,2023
...,...,...,...,...,...,...,...,...,...,...,...,...
1651,48500.0,VOLKSWAGEN,Berline,Essence,5.0,Manuelle,Traction,158000.0,12,12,November,2023
1652,110000.0,MERCEDES,Berline,Essence,10.0,Automatique,Propulsion,130000.0,4,10,November,2023
1653,115000.0,KIA,SUV/4x4,Hybride (diesel),8.0,Automatique,Traction,75000.0,12,4,November,2023
1654,58000.0,MAZDA,Berline,Essence,6.0,Manuelle,Traction,113000.0,11,7,December,2023
